# k-Nearest Neighbors (KNN)
비지도 학습에 기본

resource: https://www.analyticsvidhya.com/blog/2018/03/introduction-k-neighbours-algorithm-clustering/

    Neighbor => 사용자가 지정한 지점에 근접한 데이터 (한마디로, 이웃!)
    Class => 데이터의 분류, 즉, label
    k => neighbor 수
    
    KNN의 원리: "자신을 알기위해선 주위를 둘러봐라!"
    
    1. 사용자가 지정한 지점(test data)에 근접한 Neighbor들을 찾는다.
        - 참고: 거리를 계산하는데 있어 Euclidean Distance (유클리드 거리)를 사용한다.
    2. 이 Neighbor들이 어떻게 분류(class)되는지 알아넨다.
    3. 근접한 Neighbor들의 class로 지정된 지점의 class를 예측한다.
    4. 즉, 근접한 neighbors의 class로 일정한 지점의 class를 찾는다.
    5. 이러한 이유로 neighbors 범위 (k)가 아주 중요하다.
        - k값이 무제한적으로 올라갈 수록,
          분류모델의 분류선이 좀 더 부드러워진다 (자료 참고)
        - k값이 올라갈 수록 정확도가 떨어진다.
            - 이웃들이 한 종류로만 분류되면 괜찮지만,
              여러 분류일수록 그만큼의 에러가 있다는 뜻.
        - k값은 너무 낮아서도 안되고, 너무 커서도 안 된다.
            - 에러의 최저값이 나오는 k값을 찾아야 된다!

## import

In [1]:
import pandas as pd
import numpy as np
import math
import operator

## 학습자료 (iris라는 꽃 종류를 상대로 학습)

In [2]:
data = pd.read_csv("./data/iris/iris.csv")
data.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Name
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


## 예측할 자료

Sepal 길이, 넓이, Petal 길이, 넓이를 대상으로 하여금,
이러한 값을 가진 종류가 무엇인지 예측한다

In [5]:
testSet = [[7.2, 3.6, 5.1, 2.5]]
test = pd.DataFrame(testSet)

## Euclidean Distance (유클리드 거리)
    - 2개의 점의 거리를 계산하는데에 있어 자주 사용되는 방식이다
    - 피타고라스의 정리를 사용하여 계산
    
2개 점의 거리를 계산하는 다른 방식들:
1. Cosine similarity
2. Manhattan distance
3. Minkowski distance
4. Jaccard similarity

resource: http://dataaspirant.com/2015/04/11/five-most-popular-similarity-measures-implementation-in-python/

In [6]:
# Defining our KNN model
def KNN(trainingSet, testInstance, k):
    distances = {}
    sort = {}
    
    
    # Euclidean Distance
    dist = 0
    # 1. for each row of the training data,
    for x in range(len(trainingSet)):
        # 2. for each of the categories for both training and test data,
        for y in range(testInstance.shape[1]):
            # 3. n sum the square the difference of test and training data
            dist += np.square([testInstance[y][0] - 
                              trainingSet.iloc[x][y]][0])
        # 4. then square root of the sum (of the square of diff. of x,y)
        distances[x] = np.sqrt(dist)
        # note: dist needs to be initialized for each training data
        dist = 0
        # all this gathers the distance between
        # each of the training data and, for now, one row of the test data
    
    # Sorting them on the basis of distance (closet at the top)
    sorted_d = sorted(distances.items(), key=operator.itemgetter(1))
    
    
    # Extracting top k neighbors (closest at the top)
    neighbors = []
    for x in range(k):
        # Extracts the index of the neighbor
        neighbors.append(sorted_d[x][0])
    
    
    # Calculating the most freq class in the neighbors
    classVotes = {}
    # 1. for each of the neighbors,
    for x in range(len(neighbors)):
        # 2. find the Name of each neighbor
        response = trainingSet.iloc[neighbors[x]][-1]
        # 3. each neighbor casts a vote based on their class (Name)
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
            
    # class with the most vote at the top (descending order)
    sortedVotes = sorted(classVotes.items(),
                         key=operator.itemgetter(1), 
                         reverse=True)
    
    # return the Top Class, and neighbors
    return(sortedVotes[0][0], neighbors)

In [7]:
%%time

# Setting number of neighbors = 1
k = 3

# Running KNN model
result, neigh = KNN(data, test, k)

# Predicted class
print(result)

# Nearest neighbor(s)
print(neigh)

Iris-virginica
[141, 139, 120]
Wall time: 161 ms


## scikit_learn의 함수 사용

In [19]:
%%time

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(data.iloc[:,0:4], data['Name'])

print(neigh.predict(test))

print(neigh.kneighbors(test)[1])

['Iris-virginica']
[[141 139 120]]
Wall time: 9 ms
